In [2]:
import ee
ee.Initialize()

# Define Nairobi bounding box
nairobi = ee.Geometry.Rectangle([36.6, -1.5, 37.1, -1.1])

# Function to get Landsat image composite by year and sensor
def get_landsat_composite(year):
    start = f'{year}-01-01'
    end = f'{year}-12-31'
    
    if year >= 2014:
        collection_id = 'LANDSAT/LC08/C02/T1_L2'
        bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    elif year >= 2004:
        collection_id = 'LANDSAT/LE07/C02/T1_L2'
        bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
    else:
        collection_id = 'LANDSAT/LT05/C02/T1_L2'
        bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']

    std_band_names = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']

    collection = ee.ImageCollection(collection_id) \
        .filterBounds(nairobi) \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
        .map(lambda img: img.select(bands, std_band_names).multiply(0.0000275).add(-0.2))

    return collection.median().clip(nairobi)

# List of years to export
years = [1984, 1989, 1994, 1999, 2004, 2009, 2014, 2019, 2023]

# Loop and export images
tasks = []
for year in years:
    image = get_landsat_composite(year)
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Nairobi_Landsat_{year}',
        folder='earthengine',
        fileNamePrefix=f'nairobi_landsat_{year}',
        region=nairobi,
        scale=30,
        maxPixels=1e13
    )
    task.start()
    tasks.append((year, task))

# Optional: print task status
for year, task in tasks:
    print(f"Export task for {year}: {task.status()['state']}")


Export task for 1984: READY
Export task for 1989: READY
Export task for 1994: READY
Export task for 1999: READY
Export task for 2004: READY
Export task for 2009: READY
Export task for 2014: READY
Export task for 2019: READY
Export task for 2023: READY


In [3]:
import ee
ee.Initialize()

# Nairobi bounding box
nairobi = ee.Geometry.Rectangle([36.6, -1.5, 37.1, -1.1])

# Define fallback years and parameters
fallback_years = {
    1989: ('1988-01-01', '1990-12-31'),
    1994: ('1993-01-01', '1995-12-31'),
    1999: ('1998-01-01', '2000-12-31'),
}

for year, (start, end) in fallback_years.items():
    collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filterBounds(nairobi) \
        .filterDate(start, end) \
        .filter(ee.Filter.lt('CLOUD_COVER', 30))  # relaxed filter
    
    count = collection.size().getInfo()

    if count > 0:
        print(f"🛰️ Exporting fallback for {year} with {count} images")

        image = collection.map(
            lambda img: img.select(
                ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
                ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
            ).multiply(0.0000275).add(-0.2)
        ).median().clip(nairobi)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'Nairobi_Landsat_{year}_fallback',
            folder='earthengine',
            fileNamePrefix=f'nairobi_landsat_{year}_fallback',
            region=nairobi,
            scale=30,
            maxPixels=1e13
        )
        task.start()
    else:
        print(f"⚠️ Still no images found for {year}. Skipping fallback export.")


⚠️ Still no images found for 1989. Skipping fallback export.
🛰️ Exporting fallback for 1994 with 1 images
⚠️ Still no images found for 1999. Skipping fallback export.


In [4]:
# 1. Replace 1989 with 1988 using Landsat 5
collection_1988 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(nairobi) \
    .filterDate('1988-01-01', '1988-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 30))

if collection_1988.size().getInfo() > 0:
    image_1988 = collection_1988.map(
        lambda img: img.select(
            ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
            ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
        ).multiply(0.0000275).add(-0.2)
    ).median().clip(nairobi)

    ee.batch.Export.image.toDrive(
        image=image_1988,
        description='Nairobi_Landsat_1988_fallback',
        folder='earthengine',
        fileNamePrefix='nairobi_landsat_1988_fallback',
        region=nairobi,
        scale=30,
        maxPixels=1e13
    ).start()
else:
    print("⚠️ Still no valid image for 1988.")

# 2. Replace 1999 with Landsat 7
collection_1999 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(nairobi) \
    .filterDate('1999-05-01', '1999-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 30))

if collection_1999.size().getInfo() > 0:
    image_1999 = collection_1999.map(
        lambda img: img.select(
            ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],
            ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
        ).multiply(0.0000275).add(-0.2)
    ).median().clip(nairobi)

    ee.batch.Export.image.toDrive(
        image=image_1999,
        description='Nairobi_Landsat_1999_L7',
        folder='earthengine',
        fileNamePrefix='nairobi_landsat_1999_L7',
        region=nairobi,
        scale=30,
        maxPixels=1e13
    ).start()
else:
    print("⚠️ Still no valid image for 1999 with Landsat 7.")


⚠️ Still no valid image for 1988.
